In [ ]:
#returns 2 tables, document on ad - doc-ad normalized score ; ad strength - ad normalized score
#requires: base table of train (or validation similiar to train) with document id of display id merged (from events)
def ad_on_doc_ad_str(train):
    train_ad_count_per_display = train.groupby(['display_id'])['display_id'].agg({'ads_on_doc' : 'count'}).reset_index()
    train_temp = train.merge(train_ad_count_per_display, how = 'left', on = 'display_id')
    del train_ad_count_per_display
    ad_on_doc = train_temp.groupby(['document_id','ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
    ad_on_doc['uni_chance'] = 1 / ad_on_doc['ads_on_doc']
    ad_on_doc['clicked_percent'] = ad_on_doc['clicked'] / ad_on_doc['total']
    ad_on_doc['clicked_percent_normalized'] = (ad_on_doc['clicked'] + 12 * ad_on_doc['clicked_percent'].mean()) / (12 + ad_on_doc['total'])
    ad_on_doc['likelihood_normalized'] = ad_on_doc['clicked_percent_normalized'] / ad_on_doc['uni_chance']
    ad_on_doc['like_mul_total_normalized'] = ad_on_doc['likelihood_normalized'] * ad_on_doc['total']
    ad_on_doc = ad_on_doc.drop(ad_on_doc.columns[[2,4,5,6,7,8]],axis = 1).groupby(['document_id','ad_id']).sum().reset_index()
    ad_on_doc['score'] = ad_on_doc['like_mul_total_normalized'] / ad_on_doc['total']
    ad_on_doc.drop(ad_on_doc.columns[[2,3]],axis = 1, inplace=True)
    ad_on_doc.sort_values('score',inplace=True, ascending=False)
    ad_strength = train_temp.groupby(['ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
    ad_strength['uni_chance'] = 1 / ad_strength['ads_on_doc']
    ad_strength['clicked_percent'] = ad_strength['clicked'] / ad_strength['total']
    ad_strength['clicked_percent_normalized'] = (ad_strength['clicked'] + 12 * ad_strength['clicked_percent'].mean()) / (12 + ad_strength['total'])
    ad_strength['likelihood_normalized'] = ad_strength['clicked_percent_normalized'] / ad_strength['uni_chance']
    ad_strength['like_mul_total_normalized'] = ad_strength['likelihood_normalized'] * ad_strength['total']
    ad_strength = ad_strength.drop(ad_strength.columns[[1,3,4,5,6,7]],axis = 1).groupby(['ad_id']).sum().reset_index()
    ad_strength['score'] = ad_strength['like_mul_total_normalized'] / ad_strength['total']
    ad_strength.drop(ad_strength.columns[[1,2]],axis = 1, inplace=True)
    ad_strength.sort_values('score',inplace=True, ascending=False)
    del train_temp
    return ad_on_doc, ad_strength

In [ ]:
#scoring function calculating portion of correct display ids predictions
def score_portion(val_copy):
    max_rows = val_copy.groupby(['display_id'])['predict'].transform(max) == val_copy['predict']
    final = val_copy[max_rows]
    success = final[final['clicked'] == True]
    score = float(len(success)) / float(len(final))
    print("PORTION: %.12f" % score)
    return score

In [ ]:
#scoring function taking in consideration the distance of right ad from 1st position
def score_map(val_copy):
    val_copy.sort_values(['display_id', 'predict'], inplace=True, ascending=[True, False] )
    val_copy["seq"] = np.arange(val_copy.shape[0])
    Y_seq = val_copy[val_copy.clicked == 1].seq.values
    Y_first = val_copy[['display_id', 'seq']].drop_duplicates(subset='display_id', keep='first').seq.values
    Y_ranks = Y_seq - Y_first
    score = np.mean(1.0 / (1.0 + Y_ranks))
    print("MAP: %.12f" % score)

In [ ]:
#function giving corellation score between document and ad by its' topics and categories
def get_score(x,y,dic):
    if x > y:
        key = (y,x)
    else:
        key = (x,y)
    return dic(key)
def correlation_score(x,y,dic,x_conf,y_conf):
    return get_score(x,y,dic) * x_conf * y_conf